In [1]:
from pynq import Overlay
import numpy as np
import pandas as pd

In [2]:
from pynq import ps

print(ps.Clocks.fclk0_mhz)
ps.Clocks.fclk0_mhz = 375
print(ps.Clocks.fclk0_mhz)
print(ps.Clocks.cpu_mhz)

99.999
374.99625
1199.988


In [3]:
# Replace with your actual path
bitstream_path = "design_1.bit"
hwh_file_path = "design_1.hwh"

# Load the bitstream onto the board
overlay = Overlay(bitstream_path)
overlay.download()  # This applies the bitstream to the board

In [4]:
# Print out information about the overlay
print(overlay)

In [5]:
# Load the overlay
overlay = Overlay(bitstream_path)

# List all available IPs in the overlay
print("Available IPs in the overlay:")
for ip in overlay.ip_dict:
    print(ip)

Available IPs in the overlay:
linear_regression_0
zynq_ultra_ps_e_0


In [6]:
linear_regression_ip = overlay.linear_regression_0

In [7]:
# normalised, rgb channel intensities
X = np.array([
    [1, 0.5909, 0.4895, 0.5242],
    [1, 0.5231, 0.4349, 0.4282],
    [1, 0.7565, 0.6464, 0.6516],
    [1, 0.8476, 0.6472, 0.6024],
    [1, 0.5912, 0.6896, 0.6233],
    [1, 0.5913, 0.5117, 0.5574],
    [1, 0.6181, 0.5373, 0.5072],
    [1, 0.6512, 0.6931, 0.7386],
    [1, 0.3691, 0.3197, 0.3185],
    [1, 0.5374, 0.6470, 0.6364],
    [1, 0.6091, 0.5752, 0.5581],
    [1, 0.7883, 0.6878, 0.6064],
    [1, 0.4955, 0.5107, 0.5203],
    [1, 0.4093, 0.5500, 0.4422],
    [1, 0.6030, 0.5382, 0.5027],
    [1, 0.6738, 0.5971, 0.5125],
    [1, 0.4975, 0.4971, 0.4242],
    [1, 0.5834, 0.5893, 0.5632],
    [1, 0.4790, 0.6039, 0.5792],
    [1, 0.6776, 1.0000, 1.0000]
], dtype=np.float32)

weights = np.array([11.578635, -0.128110, -0.813338, 0.951879], dtype = np.float32)

In [8]:
from pynq import allocate
X_buffer = allocate(shape=(20, 4), dtype=np.float32)
weights_buffer = allocate(shape=(4, ), dtype=np.float32)
y_pred_buffer = allocate(shape=(20, ), dtype=np.float32)

In [9]:
np.copyto(X_buffer, X)
np.copyto(weights_buffer, weights)

In [10]:
print(X_buffer)

[[1.     0.5909 0.4895 0.5242]
 [1.     0.5231 0.4349 0.4282]
 [1.     0.7565 0.6464 0.6516]
 [1.     0.8476 0.6472 0.6024]
 [1.     0.5912 0.6896 0.6233]
 [1.     0.5913 0.5117 0.5574]
 [1.     0.6181 0.5373 0.5072]
 [1.     0.6512 0.6931 0.7386]
 [1.     0.3691 0.3197 0.3185]
 [1.     0.5374 0.647  0.6364]
 [1.     0.6091 0.5752 0.5581]
 [1.     0.7883 0.6878 0.6064]
 [1.     0.4955 0.5107 0.5203]
 [1.     0.4093 0.55   0.4422]
 [1.     0.603  0.5382 0.5027]
 [1.     0.6738 0.5971 0.5125]
 [1.     0.4975 0.4971 0.4242]
 [1.     0.5834 0.5893 0.5632]
 [1.     0.479  0.6039 0.5792]
 [1.     0.6776 1.     1.    ]]


In [11]:
print(weights_buffer)

[11.578635 -0.12811  -0.813338  0.951879]


In [12]:
linear_regression_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  X_1 = Register(X=write-only),
  X_2 = Register(X=write-only),
  y_pred_1 = Register(y_pred=write-only),
  y_pred_2 = Register(y_pred=write-only),
  weights_1 = Register(weights=write-only),
  weights_2 = Register(weights=write-only)
}

In [13]:
print(linear_regression_ip.register_map.X_1.address)
print(linear_regression_ip.register_map.y_pred_1.address)

16
28


In [14]:
linear_regression_ip.write(linear_regression_ip.register_map.X_1.address, X_buffer.device_address)
linear_regression_ip.write(linear_regression_ip.register_map.weights_1.address, weights_buffer.device_address)
linear_regression_ip.write(linear_regression_ip.register_map.y_pred_1.address, y_pred_buffer.device_address)

In [15]:
print(dir(linear_regression_ip.register_map))

['CTRL', 'GIER', 'IP_IER', 'IP_ISR', 'X_1', 'X_2', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_value', '_instances', '_map_size', '_register_classes', '_set_value', 'create_subclass', 'weights_1', 'weights_2', 'y_pred_1', 'y_pred_2']


In [16]:
# Start the IP
linear_regression_ip.register_map.CTRL.AP_START = 1

In [17]:
y_pred_buffer.sync_from_device()
print(y_pred_buffer)

[11.603782  11.565495  11.576222  11.517069  11.535325  11.617276
 11.545237  11.634543  11.574499  11.5893345 11.564015  11.495451
 11.595047  11.499784  11.542156  11.494508  11.514377  11.560694
 11.577424  11.630369 ]


In [28]:
labels_df = pd.read_csv("labels.csv", header=None)
labels = labels_df.values.flatten()

In [30]:
print(labels_df.shape)
print(X.shape)

(20, 1)
(20, 4)


In [31]:
predictions = np.array(y_pred_buffer)

In [41]:
# squared_errors = (labels - predictions) ** 2
# mse = np.mean(squared_errors)

In [42]:
# print("MSE:", mse)

In [40]:
print(predictions)

[11.603782  11.565495  11.576222  11.517069  11.535325  11.617276
 11.545237  11.634543  11.574499  11.5893345 11.564015  11.495451
 11.595047  11.499784  11.542156  11.494508  11.514377  11.560694
 11.577424  11.630369 ]


In [43]:
hi = labels  # ground truth
hi_hat = np.array(y_pred_buffer)  # predictions

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean((hi_hat - hi) ** 2))

# Pearson Correlation (r)
correlation = np.corrcoef(hi_hat, hi)[0, 1]

# Hemoglobin Estimation Accuracy (as defined)
accuracy = np.mean((hi - np.abs(hi_hat - hi)) / hi) * 100

print(f"RMSE: {rmse:.4f}")
print(f"Correlation (r): {correlation:.4f}")
print(f"Hemoglobin Estimation Accuracy: {accuracy:.2f}%")

RMSE: 1.3041
Correlation (r): 0.1112
Hemoglobin Estimation Accuracy: 90.05%
